# Apendice 10.1: Encadenamiento de Prompts

- [Leccion](#leccion)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu clave API y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

# Correccion de ruta para importar hints
import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Se ha reescrito para aceptar una lista de mensajes de longitud arbitraria
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## Leccion

El dicho dice: "Escribir es reescribir." Resulta que **Claude a menudo puede mejorar la precision de su respuesta cuando se le pide que lo haga**.

Hay muchas formas de pedirle a Claude que "piense de nuevo". Las formas que resultan naturales para pedirle a un humano que revise su trabajo tambien funcionaran generalmente con Claude. (Consulta nuestra [documentacion sobre encadenamiento de prompts](https://docs.anthropic.com/claude/docs/chain-prompts) para mas ejemplos de cuando y como usar el encadenamiento de prompts).

### Ejemplos

En este ejemplo, le pedimos a Claude que proponga diez palabras... pero una o mas de ellas no son palabras reales.

In [ ]:
# Prompt inicial
first_user = "Name ten words that all end with the exact letters 'ab'."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Almacenar e imprimir la respuesta de Claude
first_response = get_completion(messages)
print(first_response)

**Pedirle a Claude que haga su respuesta mas precisa** corrige el error.

A continuacion, hemos tomado la respuesta incorrecta de Claude de arriba y hemos agregado otro turno a la conversacion pidiendole a Claude que corrija su respuesta anterior.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

Pero, esta Claude revisando su respuesta solo porque se lo dijimos? Que pasa si empezamos con una respuesta correcta desde el principio? Perdera Claude su confianza? Aqui, hemos colocado una respuesta correcta en lugar de `first_response` y le hemos pedido que vuelva a verificar.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

Puedes notar que si generas una respuesta del bloque anterior varias veces, Claude deja las palabras tal cual la mayor parte del tiempo, pero aun ocasionalmente cambia las palabras aunque todas ya son correctas. Que podemos hacer para mitigar esto? Segun el Capitulo 8, podemos darle a Claude una salida alternativa! Intentemoslo una vez mas.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

Intenta generar respuestas del codigo anterior varias veces para ver que Claude es mucho mejor manteniendo su posicion ahora.

Tambien puedes usar el encadenamiento de prompts para **pedirle a Claude que mejore sus respuestas**. A continuacion, le pedimos a Claude que primero escriba una historia, y luego que mejore la historia que escribio. Tus gustos personales pueden variar, pero muchos podrian estar de acuerdo en que la segunda version de Claude es mejor.

Primero, generemos la primera version de la historia de Claude.

In [ ]:
# Prompt inicial
first_user = "Write a three-sentence short story about a girl who likes to run."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Almacenar e imprimir la respuesta de Claude
first_response = get_completion(messages)
print(first_response)

Ahora hagamos que Claude mejore su primer borrador.

In [ ]:
second_user = "Make the story better."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

Esta forma de sustitucion es muy poderosa. Hemos estado usando marcadores de sustitucion para pasar listas, palabras, respuestas anteriores de Claude, y mas. Tambien puedes **usar la sustitucion para hacer lo que llamamos "llamada a funciones" (function calling), que consiste en pedirle a Claude que ejecute alguna funcion, y luego tomar los resultados de esa funcion y pedirle a Claude que haga aun mas con los resultados**. Funciona como cualquier otra sustitucion. Mas sobre esto en el siguiente apendice.

A continuacion hay un ejemplo mas de tomar los resultados de una llamada a Claude e insertarlos en otra llamada mas larga. Comencemos con el primer prompt (que incluye prellenado de la respuesta de Claude esta vez).

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Almacenar e imprimir la respuesta de Claude
first_response = get_completion(messages)
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(first_response)

Pasemos esta lista de nombres a otro prompt.

In [ ]:
second_user = "Alphabetize the list."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

Ahora que has aprendido sobre el encadenamiento de prompts, ve al Apendice 10.2 para aprender como implementar la llamada a funciones usando encadenamiento de prompts.

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompts mostrados en esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
# Prompt inicial
first_user = "Name ten words that all end with the exact letters 'ab'."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Almacenar e imprimir la respuesta de Claude
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

In [ ]:
# Prompt inicial
first_user = "Write a three-sentence short story about a girl who likes to run."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Almacenar e imprimir la respuesta de Claude
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Almacenar e imprimir la respuesta de Claude
first_response = get_completion(messages)
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# Array de mensajes para la API
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Imprimir la respuesta de Claude
print("------------------------ Array completo de mensajes con sustituciones de variables ------------------------")
print(messages)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(messages))